In [ ]:
!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.
!pip install pyquaternion

In [ ]:
from IPython.display import display, Image
from IPython import display
import os
import shutil
from tqdm import tqdm
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from pyquaternion import Quaternion

In [ ]:
%matplotlib inline
#from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/content/drive/MyDrive/centerfusion/CenterFusion/data/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 5.569 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [ ]:
output_dir = '/content/drive/MyDrive/TestdataDetection'
image_output_dir_train = os.path.join(output_dir, 'images/train')
label_output_dir_train = os.path.join(output_dir, 'labels/train')
image_output_dir_val = os.path.join(output_dir, 'images/val')
label_output_dir_val = os.path.join(output_dir, 'labels/val')

In [ ]:
#os.makedirs(image_output_dir_train, exist_ok=True)
#os.makedirs(label_output_dir_train, exist_ok=True)
os.makedirs(image_output_dir_val, exist_ok=True)
os.makedirs(label_output_dir_val, exist_ok=True)

In [ ]:
class_names = [
    'animal', 'human','movable_object','static_object','vehicle']

# **Functional** **Code**

In [ ]:
def get_yolo_format_bbox(box,size, img_width, img_height):
    """Convert the nuScenes box to YOLO format"""
    x_center = box[0] / img_width
    y_center = box[1] / img_height
    width = size [0] / img_width
    height = size [1] / img_height
    return [x_center, y_center, width, height]

In [ ]:
def process_scenes(scenes, image_output_dir, label_output_dir):
    for scene in tqdm(scenes):
        first_sample_token = scene['first_sample_token']
        sample = nusc.get('sample', first_sample_token)

        while sample:
            processed_images = set()
            for data_key in ['CAM_FRONT']: #, 'CAM_BACK', 'CAM_FRONT_LEFT', 'CAM_FRONT_RIGHT', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']:
                sample_data_token = sample['data'][data_key]
                data_path, boxes, camera_intrinsic = nusc.get_sample_data(sample_data_token)
                image_basename = os.path.basename(data_path)

                if image_basename not in processed_images:
                    image_output_path = os.path.join(image_output_dir, image_basename)
                    shutil.copy(data_path, image_output_path)
                    processed_images.add(image_basename)

                imsize = (1600, 900)
                label_file_path = os.path.join(label_output_dir, image_basename.replace('.jpg', '.txt'))

                with open(label_file_path, 'w') as label_file:
                    for box in boxes:
                        category = box.name.split('.')[0]
                        class_id = class_names.index(category)

                        # Project the corners of the 3D bounding box to 2D
                        corners_3d = box.corners()
                        corners_2d = view_points(corners_3d, camera_intrinsic, normalize=True)[:2, :]
                        min_x, max_x = max(0,min(corners_2d[0])), min(1600,max(corners_2d[0]))
                        min_y, max_y = max(0,min(corners_2d[1])), min(900,max(corners_2d[1]))

                        width_2d = max_x - min_x
                        height_2d = max_y - min_y

                        center_x = (min_x + max_x) / 2
                        center_y = (min_y + max_y) / 2

                        bbox_yolo = get_yolo_format_bbox([center_x, center_y], [width_2d, height_2d], 1600, 900)

                        label_file.write(f"{class_id} {' '.join(map(str, bbox_yolo))}\n")
                        print(f"{class_id} {' '.join(map(str, bbox_yolo))}")

            next_sample_token = sample['next']
            if next_sample_token:
                sample = nusc.get('sample', next_sample_token)
            else:
                break


In [ ]:
# Split scenes into train and val
import random

# Define the ratio of validation scenes
validation_ratio = 0.9  # 20% of scenes for validation, 80% for training

# Get the list of all scenes
all_scenes = nusc.scene

# Shuffle the scenes randomly
random.shuffle(all_scenes)

# Calculate the number of scenes for validation
num_validation_scenes = int(validation_ratio * len(all_scenes))

# Split scenes into validation and training sets
val_scenes = all_scenes[:num_validation_scenes]
train_scenes = all_scenes[num_validation_scenes:]

#val_scenes
#train_scenes

In [ ]:
total_scenes = nusc.scene
test_scenes = total_scenes[7:8]
test_scenes

[{'token': 'd25718445d89453381c659b9c8734939',
  'log_token': '8fefc430cbfa4c2191978c0df302eb98',
  'nbr_samples': 41,
  'first_sample_token': '5998b71b64c146769bde1d5430741381',
  'last_sample_token': 'ee9b3f38dadd40588f43ba590c4751b9',
  'name': 'scene-1077',
  'description': 'Night, big street, bus stop, high speed, construction vehicle'}]

In [ ]:
total_scenes

[{'token': 'cc8c0bf57f984915a77078b10eb33198',
  'log_token': '7e25a2c8ea1f41c5b0da1e69ecfa71a2',
  'nbr_samples': 39,
  'first_sample_token': 'ca9a282c9e77460f8360f564131a8af5',
  'last_sample_token': 'ed5fc18c31904f96a8f0dbb99ff069c0',
  'name': 'scene-0061',
  'description': 'Parked truck, construction, intersection, turn left, following a van'},
 {'token': 'fcbccedd61424f1b85dcbf8f897f9754',
  'log_token': '53cf9c55dd8644bea67b9f009fc1ee38',
  'nbr_samples': 40,
  'first_sample_token': '3e8750f331d7499e9b5123e9eb70f2e2',
  'last_sample_token': '281b92269fd648d4b52d06ac06ca6d65',
  'name': 'scene-0103',
  'description': 'Many peds right, wait for turning car, long bike rack left, cyclist'},
 {'token': '6f83169d067343658251f72e1dd17dbc',
  'log_token': '881dd2f8aaec49b681eb54be5bf3b3e2',
  'nbr_samples': 41,
  'first_sample_token': '8687ba92abd3406aa797115b874ebeba',
  'last_sample_token': 'dcbe451d383e450786aaad04ab9d3790',
  'name': 'scene-0553',
  'description': 'Wait at intersect

In [ ]:
# Process train scenes
#process_scenes(train_scenes, image_output_dir_train, label_output_dir_train)

# Process val scenes
process_scenes(test_scenes, image_output_dir_val, label_output_dir_val)

  0%|          | 0/1 [00:00<?, ?it/s]

4 0.23123906001922598 0.5722692980812448 0.07233251592604847 0.05315326169913868
4 0.32619238948038853 0.5974261616867678 0.1342069074469466 0.09565001730885109
4 0.05359339311491934 0.583377125828006 0.10718678622983868 0.07650840333256016
4 0.09373622703181286 0.6543476027755522 0.18747245406362573 0.21726690231126145
4 0.1280791938935476 0.5738775938869576 0.07863576044583294 0.06037048665591467
4 0.4985907124772309 0.5665885933538334 0.08830085117678876 0.06542009710723032
4 0.2128289332046729 0.5716620137651433 0.04710959905973265 0.043364281187967195
4 0.2929119964814139 0.5543503322653028 0.04015538621884183 0.03945256044232009
4 0.6046249748946174 0.5862330533165634 0.17759608052236758 0.1259577434969003
4 0.21304294463950899 0.5573209407715034 0.07453869015278407 0.06384921353030809
4 0.16796194226258163 0.6409862911961887 0.17000955035331175 0.19269950907868394
4 0.35988220946467125 0.5672160139656535 0.0812875688840748 0.06724047307096946
4 0.829655129562627 0.57284933412999

100%|██████████| 1/1 [00:15<00:00, 15.92s/it]

4 0.7904833581384814 0.5696360257003554 0.04881611025857325 0.04707414661867782
4 0.5915948172039678 0.5668017987679521 0.01410016076388473 0.019924005282828717
4 0.5307751910002828 0.5971691686109684 0.08366622805294334 0.11853448462499544
4 0.7221993406795929 0.5638263562018858 0.03512931413472714 0.038404683616854146
